# Ноутбук для подбора количества кофе, которое минимизирует стресс

## main class

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gamma, norm


class HandsTable():
    minimize = True
    rho = 3.37

    def __init__(self, options_list, minimize=True, rho=1.0):
        """инициализация

        Args:
            options_list (list): список из названий
            minimize (bool, optional): Минимизировать если True, максимизировать если False. Defaults to True.
            rho (float, optional): Риск толерантность, чем больше, тем больше риска готовы принять. Defaults to 1.0.
        """
        self.hands = pd.DataFrame({'name': options_list,
                                   'mu': 0.0,
                                   'Te': 0.0,
                                   'alpha': 0.5,
                                   'beta': 0.5
                                   })
        self.minimize = minimize
        if rho is not None:
            self.rho = rho

        self.history = pd.DataFrame(columns=['option', 'value'])

    @classmethod
    def to_minutes(self, timestr: str):
        '''
        convert timestr to float minutes
        '''
        return pd.to_timedelta(timestr).total_seconds()/60

    @classmethod
    def update_mean(self, X, T_last, mu_last):
        mu_new = T_last * mu_last / (T_last + 1) + X / (T_last + 1)
        return mu_new

    @classmethod
    def update_samples(self, T):
        return T + 1

    @classmethod
    def update_shape(self, a):
        return a + 0.5

    @classmethod
    def update_rate(self, X, mu_last, beta_last, T_last):
        beta_new = beta_last + (T_last / (T_last + 1)) * \
            (np.square(X - mu_last)) / 2
        return beta_new

    def update_hands(self, name, value):
        if isinstance(value, str):
            try:
                value = HandsTable.to_minutes(value)
            except ValueError:
                raise ValueError('input time string in hh:mm:ss format')
        elif isinstance(value, float) or isinstance(value, int):
            pass
        else:
            raise ValueError('input time string or int/float value')

        _, mu, t, alpha, beta = self.hands[self.hands.name == name].values[0]
        beta = HandsTable.update_rate(value, mu, beta, t)
        mu = HandsTable.update_mean(value, t, mu)
        t = HandsTable.update_samples(t)
        alpha = HandsTable.update_shape(alpha)

        # added code to write history
        self.history.loc[len(self.history.index)] = [name, value]

        self.hands.loc[self.hands.name == name, 'mu'] = mu
        self.hands.loc[self.hands.name == name, 'Te'] = t
        self.hands.loc[self.hands.name == name, 'alpha'] = alpha
        self.hands.loc[self.hands.name == name, 'beta'] = beta

    def grade(self):
        hands_output = self.hands.copy()
        tau = gamma.rvs(a=hands_output.alpha, scale=1/hands_output.beta)
        theta_drops = norm.rvs(hands_output.mu, 1/hands_output.Te)
        hands_output['tau'] = tau
        hands_output['theta'] = theta_drops
        hands_output['SD'] = np.sqrt(1/tau)

        if self.minimize == True:
            hands_output['var95'] = theta_drops + \
                norm.ppf(1-0.05/2) * hands_output.SD
            if hands_output.mu.min() == 0:

                output_df = hands_output.reindex(np.argsort(hands_output.Te))
            else:
                output_df = hands_output.reindex(
                    np.argsort(self.rho * theta_drops + 1/tau))
        else:
            hands_output['var95'] = theta_drops + \
                norm.ppf(0.05/2) * hands_output.SD
            if hands_output.mu.min() == 0:
                output_df = hands_output.reindex(np.argsort(hands_output.Te))
            else:
                output_df = hands_output.reindex(
                    np.argsort(self.rho * theta_drops - 1/tau)[::-1])

        return output_df

    def __str__(self):
        return repr(self.hands)

# Применение

In [2]:
cups = HandsTable(options_list=[*range(4+1)], minimize=True)
cups.update_hands(0, 32) # суббота 9 ноября
cups.update_hands(2, 33) # воскресенье 10 ноября
cups.update_hands(1, 35) # вторник 12 ноября
cups.update_hands(3, 34) # среда 13 ноября
cups.update_hands(4, 34) # четверг 14 ноября
cups.update_hands(3, 34) # пятница 15 ноября
cups.update_hands(1, 34) # суббота 16 ноября
cups.update_hands(1, 43) # воскресенье 17 ноября
# cups.update_hands(6, 28) # понедельник 18 ноября я убрал 5  и 6 из подбора, это мучительно
# cups.update_hands(5, 39) # вторник 19 ноября
cups.update_hands(4, 38) # wednesday 20 ноября
cups.update_hands(0, 36) # thursday 21 ноября
cups.update_hands(2, 33) # friday 22 ноября
cups.update_hands(0, 37) # sat 23 ноября
cups.update_hands(1, 32) # sun 24 ноября
cups.update_hands(2, 28) # mon 25 ноября
cups.update_hands(3, 29) # tue 26 ноября
cups.update_hands(3, 25) # wed 27 ноября
cups.update_hands(2, 30) # th 28 ноября
# cups.update_hands(2, 30) # fr 29 ноября
# cups.update_hands(2, 80) # sat 30 ноября
# cups.update_hands(2, 80) # sun 1 dec

cups.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
4,4,36.0,2.0,1.5,4.5,0.416933,35.652766,1.548698,38.688159
0,0,35.0,3.0,2.0,7.5,0.265620,34.809084,1.940305,38.612011
2,2,31.0,4.0,2.5,9.5,0.106850,30.620364,3.059230,36.616345
3,3,30.5,4.0,2.5,29.0,0.088997,30.850110,3.352070,37.420046
1,1,36.0,4.0,2.5,35.5,0.011206,35.530833,9.446684,54.045994


In [3]:
cups.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
0,0,35.0,3.0,2.0,7.5,1.101842,35.332193,0.952665,37.199382
4,4,36.0,2.0,1.5,4.5,0.286954,36.801756,1.866784,40.460585
2,2,31.0,4.0,2.5,9.5,0.061745,31.186168,4.024370,39.073788
1,1,36.0,4.0,2.5,35.5,0.080862,35.852496,3.516629,42.744963
3,3,30.5,4.0,2.5,29.0,0.027933,30.589804,5.983302,42.316861


In [4]:
import plotly.express as px

fig = px.violin(cups.history, y="value", x="option", box=True, points="all",
          hover_data=cups.history.columns)
fig.show()

# Какой выход из МЦК утром?
* Садиться в ту дверь, которая кажется ближе к выходу
* считать от выхода на платформу до выхода из крутящихся дверей офиса


In [5]:
morning_exit = HandsTable(options_list=["мцк","мцд"], minimize=True)
# 3 мцк
# 6 мцд
morning_exit.update_hands("мцк", '0:08:03.7') # вторник 19 ноября
morning_exit.update_hands("мцд", '0:08:40') # среда 20 ноября
morning_exit.update_hands("мцк", '0:09:23.4') # mon 25 ноября
morning_exit.update_hands("мцд", '0:07:46.5') # вторник 26 ноября
morning_exit.update_hands("мцд", '0:07:27.9') # среда 27 ноября
morning_exit.update_hands("мцк", '0:09:11') # пятница 29 ноября

morning_exit.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
1,мцд,7.968889,3.0,2.0,0.889195,2.104167,7.440364,0.689382,8.791528
0,мцк,8.878333,3.0,2.0,1.010886,2.187720,9.230249,0.676089,10.555359


In [6]:
fig = px.violin(morning_exit.history, y="value", x="option", box=True, points="all",
          hover_data=cups.history.columns)
fig.show()

# Какой вход в МЦК вечером?
* Садиться в ту дверь, которая кажется ближе к выходу
* считать от выхода из крутящихся дверей до посадки в поезд


In [7]:
evening_entrance = HandsTable(options_list=["мцк","мцд"], minimize=True)
# 3 мцк
# 6 мцд
evening_entrance.update_hands("мцд", '0:08:03.7') # четверг 21 ноября
evening_entrance.update_hands("мцк", '0:11:14') # четверг 28 ноября

evening_entrance.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
1,мцд,8.061667,1.0,1.0,0.5,3.564792,8.989594,0.529643,10.027674
0,мцк,11.233333,1.0,1.0,0.5,7.157232,10.251015,0.373790,10.983629


In [8]:
fig = px.violin(evening_entrance.history, y="value", x="option", box=True, points="all",
          hover_data=cups.history.columns)
fig.show()